# Combined Experiments

first do the experiment1 ,the combined data, and then split it into 5 folds, then organize them into five pairs of training and testing data

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

# Load the combined data
combined_data_42 = pd.read_csv('combined_data_42.csv')

# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create storage for training and test data splits
folds = []

# Split the data into 5 folds
for train_index, test_index in kf.split(combined_data_42):
    train_data = combined_data_42.iloc[train_index]
    test_data = combined_data_42.iloc[test_index]
    folds.append((train_data, test_data))

# Save each training and test group as separate files
for i, (train_data, test_data) in enumerate(folds):
    train_data.to_csv(f'combined_train_42_fold{i+1}.csv', index=False)
    test_data.to_csv(f'combined_test_42_fold{i+1}.csv', index=False)

print("Training and test data for 5 folds with seed 42 have been generated and saved.")


Training and test data for 5 folds with seed 42 have been generated and saved.


do the feature selection for each group

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Function to perform feature selection
def feature_selection(train_data, target_column, preselected_features, k=25):
    # Separate features and target
    X = train_data.drop(columns=[target_column, 'TIMESTAMP'], errors='ignore')  # Drop TIMESTAMP
    y = train_data[target_column]

    # Train a RandomForestRegressor model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=100)
    rf_model.fit(X, y)

    # Get feature names and their importance scores
    feature_importances = rf_model.feature_importances_
    feature_names = X.columns

    # Combine feature names and scores into a DataFrame
    selected_features = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)

    # Select the preselected features (label, T_out, RH_out)
    preselected_df = selected_features[selected_features['Feature'].isin(preselected_features)]

    # Select the remaining top features after excluding preselected ones
    remaining_features = selected_features[~selected_features['Feature'].isin(preselected_features)]
    top_remaining_features = remaining_features.head(k - len(preselected_features))

    # Combine preselected features with the top remaining features
    final_features = pd.concat([preselected_df, top_remaining_features])

    return final_features

# Perform feature selection for each fold
target_column = 'WH_RTU_Total'
preselected_features = ['label', 'T_out', 'RH_out']

for i in range(1, 6):
    train_data = pd.read_csv(f'combined_train_42_fold{i}.csv')

    # Perform feature selection
    top_features = feature_selection(train_data, target_column, preselected_features, k=25)

    # Save top features
    top_features.to_csv(f'combined_train_42_fold{i}_top25_features.csv', index=False)

    print(f"Top 25 features for fold {i} with seed 42 saved.")


Top 25 features for fold 1 with seed 42 saved.
Top 25 features for fold 2 with seed 42 saved.
Top 25 features for fold 3 with seed 42 saved.
Top 25 features for fold 4 with seed 42 saved.
Top 25 features for fold 5 with seed 42 saved.


then i need to select the related data from both the training and test set in the five groups based on the feature selected in former section(which is the training data)

In [ ]:
# Target variable
target_column = 'WH_RTU_Total'

for i in range(1, 6):
    # Load the original training and test datasets for the current fold
    train_data = pd.read_csv(f'combined_train_42_fold{i}.csv')
    test_data = pd.read_csv(f'combined_test_42_fold{i}.csv')

    # Load the top 25 features selected from the current fold
    top_features = pd.read_csv(f'combined_train_42_fold{i}_top25_features.csv')['Feature'].tolist()

    # Ensure the target column is included in the selected features
    train_columns = top_features + [target_column]  # Train data includes target
    test_columns = top_features + [target_column]  # Test data includes target

    # Filter the training data
    filtered_train_data = train_data[train_columns]
    filtered_train_data.to_csv(f'fold_42_{i}_filtered_train_data.csv', index=False)
    print(f"Filtered train data for fold {i} with seed 42 saved.")

    # Filter the test data
    filtered_test_data = test_data[test_columns]

    # Split the test data into heating and cooling datasets
    test_data_heating = filtered_test_data[filtered_test_data['label'] == 1]
    test_data_cooling = filtered_test_data[filtered_test_data['label'] == 0]

    # Save all datasets
    filtered_test_data.to_csv(f'fold_42_{i}_filtered_test_data.csv', index=False)
    test_data_heating.to_csv(f'fold_42_{i}_filtered_test_data_heating.csv', index=False)
    test_data_cooling.to_csv(f'fold_42_{i}_filtered_test_data_cooling.csv', index=False)
    print(f"Filtered test data, heating data, and cooling data for fold {i} with seed 42 saved.")


Filtered train data for fold 1 with seed 42 saved.
Filtered test data, heating data, and cooling data for fold 1 with seed 42 saved.
Filtered train data for fold 2 with seed 42 saved.
Filtered test data, heating data, and cooling data for fold 2 with seed 42 saved.
Filtered train data for fold 3 with seed 42 saved.
Filtered test data, heating data, and cooling data for fold 3 with seed 42 saved.
Filtered train data for fold 4 with seed 42 saved.
Filtered test data, heating data, and cooling data for fold 4 with seed 42 saved.
Filtered train data for fold 5 with seed 42 saved.
Filtered test data, heating data, and cooling data for fold 5 with seed 42 saved.


generate the mae and R2 value for the four models:linear regression, random forest ,xgboost and stacking ensemble regressor.

In [ ]:
!pip install pandas numpy scikit-learn xgboost

from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results = []


# Loop through each fold
for i in range(1, 6):
    # Load filtered training and test datasets for the current fold
    train_data = pd.read_csv(f'fold_42_{i}_filtered_train_data.csv')
    test_data_combined = pd.read_csv(f'fold_42_{i}_filtered_test_data.csv')
    test_data_cooling = pd.read_csv(f'fold_42_{i}_filtered_test_data_cooling.csv')
    test_data_heating = pd.read_csv(f'fold_42_{i}_filtered_test_data_heating.csv')

    # Separate features and target for training data
    target_column = 'WH_RTU_Total'
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]

    # Define models with default parameters
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(random_state=100),
        'XGBoost': XGBRegressor(random_state=100),
        'Stacking Regressor': StackingRegressor(
            estimators=[
                ('rf', RandomForestRegressor(random_state=100)),  # Random Forest
                ('gb', GradientBoostingRegressor(random_state=100))  # Gradient Boosting
            ],
            final_estimator=LinearRegression()  # Linear Regression as the meta-learner
        )
    }

    # Function to evaluate models on a given dataset
    def evaluate_model(model, X_train, y_train, X_test, y_test, dataset_type):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        return mae, r2

    # Evaluate models on combined, cooling, and heating datasets
    for model_name, model in models.items():
        for dataset_type, test_data in [('Combined', test_data_combined),
                                        ('Cooling', test_data_cooling),
                                        ('Heating', test_data_heating)]:
            # Separate features and target for the current test dataset
            X_test = test_data.drop(columns=[target_column])
            y_test = test_data[target_column]

            # Calculate MAE and R²
            mae, r2 = evaluate_model(model, X_train, y_train, X_test, y_test, dataset_type)

            # Store results
            results.append({
                'Fold': i,
                'Seed': 42,  # Add the seed column
                'Model': model_name,
                'Dataset': dataset_type,
                'MAE': mae,
                'R²': r2,
                'Experiment': 1  # Add the experiment column
            })

            print(f"Fold {i}, Model {model_name}, Dataset {dataset_type}, MAE: {mae:.4f}, R²: {r2:.4f}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv(f'model_results_42_mae_r2_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for All Models:")
summary = results_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).sort_values(by=['Model', 'Dataset'])
print(summary)


Fold 1, Model Linear Regression, Dataset Combined, MAE: 4.4129, R²: 0.6871
Fold 1, Model Linear Regression, Dataset Cooling, MAE: 4.8859, R²: 0.4429
Fold 1, Model Linear Regression, Dataset Heating, MAE: 3.8922, R²: 0.3031
Fold 1, Model Random Forest, Dataset Combined, MAE: 2.8386, R²: 0.8362
Fold 1, Model Random Forest, Dataset Cooling, MAE: 2.5654, R²: 0.7507
Fold 1, Model Random Forest, Dataset Heating, MAE: 3.1393, R²: 0.5090
Fold 1, Model XGBoost, Dataset Combined, MAE: 2.7947, R²: 0.8374
Fold 1, Model XGBoost, Dataset Cooling, MAE: 2.4817, R²: 0.7626
Fold 1, Model XGBoost, Dataset Heating, MAE: 3.1391, R²: 0.4830
Fold 1, Model Stacking Regressor, Dataset Combined, MAE: 2.9883, R²: 0.8367
Fold 1, Model Stacking Regressor, Dataset Cooling, MAE: 2.7672, R²: 0.7542
Fold 1, Model Stacking Regressor, Dataset Heating, MAE: 3.2317, R²: 0.5026
Fold 2, Model Linear Regression, Dataset Combined, MAE: 4.5668, R²: 0.6846
Fold 2, Model Linear Regression, Dataset Cooling, MAE: 5.2644, R²: 0.364

## Random Forest

### Random Forest Hyperparameter Tuning python



In [ ]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_rf = []



# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold_42_{i}_filtered_train_data.csv')
    test_data_combined = pd.read_csv(f'fold_42_{i}_filtered_test_data.csv')
    test_data_cooling = pd.read_csv(f'fold_42_{i}_filtered_test_data_cooling.csv')
    test_data_heating = pd.read_csv(f'fold_42_{i}_filtered_test_data_heating.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Initialize Random Forest and perform GridSearchCV
    rf = RandomForestRegressor(random_state=100)
    grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_rf.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_rf = grid_search_rf.best_estimator_
    best_params = grid_search_rf.best_params_

    # Evaluate on Combined, Cooling, and Heating datasets
    for dataset_type, test_data in [('Combined', test_data_combined),
                                    ('Cooling', test_data_cooling),
                                    ('Heating', test_data_heating)]:
        # Split test features and target
        X_test = test_data.drop(columns=['WH_RTU_Total'])
        y_test = test_data['WH_RTU_Total']

        # Predict and calculate metrics
        y_pred = best_rf.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results_rf.append({
            'Fold': i,
            'Seed': 42,  # Add the seed column
            'Model': 'Random Forest',
            'Dataset': dataset_type,
            'MAE': mae,
            'R²': r2,
            'Best Params': best_params,
            'Experiment': 1
        })

        print(f"Fold {i}, Model Random Forest, Dataset {dataset_type}, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_rf_df = pd.DataFrame(results_rf)

# Save detailed results to CSV
results_rf_df.to_csv(f'random_forest_42_combined_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for Random Forest:")
summary_rf = results_rf_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_rf)

# Save summary to CSV 这个section里面有没有inner seed呢？
summary_rf.to_csv(f'random_forest_42_combined_hyperparameter_results_summary.csv', index=False)


Fold 1, Model Random Forest, Dataset Combined, MAE: 2.8406, R²: 0.8350, Best Params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 150}
Fold 1, Model Random Forest, Dataset Cooling, MAE: 2.5471, R²: 0.7490, Best Params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 150}
Fold 1, Model Random Forest, Dataset Heating, MAE: 3.1636, R²: 0.5054, Best Params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 150}
Fold 2, Model Random Forest, Dataset Combined, MAE: 2.7867, R²: 0.8436, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}
Fold 2, Model Random Forest, Dataset Cooling, MAE: 2.4041, R²: 0.7353, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}
Fold 2, Model Random Forest, Dataset Heating, MAE: 3.1731, R²: 0.5455, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}
Fold 3, Model Random Forest, Dataset Combined, MAE: 2.7254, R²: 0.8499, Best Params: {'max_depth': 10, 'min_sa

### random forest feature importance generation

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Placeholder for feature importance results
feature_importance_results = []

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold_42_{fold}_filtered_train_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model for the current fold
    results_rf_df = pd.read_csv(f'random_forest_42_combined_hyperparameter_results_detailed.csv')
    best_params = results_rf_df.loc[(results_rf_df['Fold'] == fold) & (results_rf_df['Dataset'] == 'Combined'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)
    best_rf = RandomForestRegressor(random_state=100, **best_params_dict)
    best_rf.fit(X_train, y_train)

    # Calculate feature importance
    feature_importances = best_rf.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances,
        'Fold': fold,
        'Seed': 42,  # Add seed column
        'Model': 'Random Forest',
        'Experiment': 1 # Add model column
    })
    feature_importance_results.append(importance_df)

# Combine and save feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)
feature_importance_combined.to_csv(f'random_forest_42_combined_feature_importance.csv', index=False)

print(f"Feature importance for seed 42 has been successfully saved.")


Feature importance for seed 42 has been successfully saved.


### generate residual ,predict, label, tout and rh out

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Fixed seed value

merged_data = []  # Placeholder for storing final merged data

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold_42_{fold}_filtered_train_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    best_params_df = pd.read_csv('random_forest_42_combined_hyperparameter_results_detailed.csv')
    best_params = best_params_df.loc[(best_params_df['Fold'] == fold) & (best_params_df['Dataset'] == 'Combined'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train Random Forest model
    best_rf = RandomForestRegressor(random_state=100, **best_params_dict)
    best_rf.fit(X_train, y_train)

    # Loop through Cooling and Heating datasets
    for dataset_type in ['cooling', 'heating']:
        # Load test data
        test_data = pd.read_csv(f'fold_42_{fold}_filtered_test_data_{dataset_type}.csv')

        # Split features and target
        X_test = test_data.drop(columns=['WH_RTU_Total'])
        y_test = test_data['WH_RTU_Total']

        # Predict and calculate residuals
        y_pred = best_rf.predict(X_test)
        test_data['Predicted'] = y_pred
        test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

        # Select and rename required columns
        if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
            selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'label']].rename(columns={
                'WH_RTU_Total': 'Actual'
            })
        else:
            raise ValueError(f"'T_out' or 'RH_out' is missing in test data for fold {fold} and dataset {dataset_type}")

        # Add fold, seed, and model columns
        selected_data['Fold'] = fold
        selected_data['Seed'] = 42
        selected_data['Model'] = 'Random Forest'
        selected_data['Experiment'] = 1

        # Append to final merged data
        merged_data.append(selected_data)
        print(f"Processed fold {fold}, dataset {dataset_type}.")

# Combine all folds and datasets into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)
# Save the merged data
merged_final_data.to_csv(f'random_forest_42_combined_residual.csv', index=False)

print("All folds processed and merged data saved successfully.")


Processed fold 1, dataset cooling.
Processed fold 1, dataset heating.
Processed fold 2, dataset cooling.
Processed fold 2, dataset heating.
Processed fold 3, dataset cooling.
Processed fold 3, dataset heating.
Processed fold 4, dataset cooling.
Processed fold 4, dataset heating.
Processed fold 5, dataset cooling.
Processed fold 5, dataset heating.
All folds processed and merged data saved successfully.


## XGBoost

### XGBoost Hyperparameter Tuning python



In [ ]:
!pip install pandas numpy scikit-learn xgboost
!pip install scikit-learn==1.0.2
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_xgb = []



# Define hyperparameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold_42_{i}_filtered_train_data.csv')
    test_data_combined = pd.read_csv(f'fold_42_{i}_filtered_test_data.csv')
    test_data_cooling = pd.read_csv(f'fold_42_{i}_filtered_test_data_cooling.csv')
    test_data_heating = pd.read_csv(f'fold_42_{i}_filtered_test_data_heating.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Initialize XGBoost and perform GridSearchCV
    xgb = XGBRegressor(random_state=100)
    grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_xgb.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_xgb = grid_search_xgb.best_estimator_
    best_params = grid_search_xgb.best_params_

    # Evaluate on Combined, Cooling, and Heating datasets
    for dataset_type, test_data in [('Combined', test_data_combined),
                                    ('Cooling', test_data_cooling),
                                    ('Heating', test_data_heating)]:
        # Split test features and target
        X_test = test_data.drop(columns=['WH_RTU_Total'])
        y_test = test_data['WH_RTU_Total']

        # Predict and calculate metrics
        y_pred = best_xgb.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results_xgb.append({
            'Fold': i,
            'Seed': 42,
            'Model': 'XGBoost',
            'Dataset': dataset_type,
            'MAE': mae,
            'R²': r2,
            'Best Params': best_params,
            'Experiment': 1
        })

        print(f"Fold {i}, Model XGBoost, Dataset {dataset_type}, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_xgb_df = pd.DataFrame(results_xgb)

# Save detailed results to CSV
results_xgb_df.to_csv(f'xgboost_42_combined_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for XGBoost:")
summary_xgb = results_xgb_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_xgb)

# Save summary to CSV
summary_xgb.to_csv(f'xgboost_42_combined_hyperparameter_results_summary.csv', index=False)


Fold 1, Model XGBoost, Dataset Combined, MAE: 2.7525, R²: 0.8469, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 1, Model XGBoost, Dataset Cooling, MAE: 2.3866, R²: 0.7826, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 1, Model XGBoost, Dataset Heating, MAE: 3.1553, R²: 0.4951, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 2, Model XGBoost, Dataset Combined, MAE: 2.7602, R²: 0.8493, Best Params: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 50, 'subsample': 0.8}
Fold 2, Model XGBoost, Dataset Cooling, MAE: 2.3681, R²: 0.7570, Best Params: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 50, 'subsample': 0.8}
Fold 2, Model XGBoost, Dataset Heating, MAE: 3.1563, R²: 0.5324, Best Params: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 50, 'subsample': 0.8}
Fold 3, Model XGBoost, Dataset Combined, MAE: 2.7552, R²: 0.8508,

### feature importance for xgboost

In [ ]:
import pandas as pd
from xgboost import XGBRegressor

# Placeholder for feature importance results
feature_importance_results = []



# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold_42_{fold}_filtered_train_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Extract the best parameters from the results file
    results_xgb_df = pd.read_csv(f'xgboost_42_combined_hyperparameter_results_detailed.csv')
    best_params = results_xgb_df.loc[
        (results_xgb_df['Fold'] == fold) & (results_xgb_df['Dataset'] == 'Combined'),
        'Best Params'
    ].iloc[0]
    best_params_dict = eval(best_params)  # Parse string into dictionary

    # Initialize and train the model with the best parameters
    best_xgb = XGBRegressor(random_state=100, **best_params_dict)
    best_xgb.fit(X_train, y_train)

    # Calculate feature importance
    feature_importances = best_xgb.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances,
        'Fold': fold,
        'Seed': 42,  # Add seed column
        'Model': 'XGBoost',
        'Experiment': 1  # Add model column
    })
    feature_importance_results.append(importance_df)

# Combine all feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)

# Save the feature importance results to a file
feature_importance_combined.to_csv(f'xgboost_42_combined_feature_importance.csv', index=False)

print(f"Feature importance for seed 42 has been successfully saved.")


Feature importance for seed 42 has been successfully saved.


### generate residual，predict，label，tout and rh out

In [ ]:
import pandas as pd
from xgboost import XGBRegressor

# Fixed seed value

merged_data = []  # Placeholder for storing final merged data

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold_42_{fold}_filtered_train_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    best_params_df = pd.read_csv('xgboost_42_combined_hyperparameter_results_detailed.csv')
    best_params = best_params_df.loc[(best_params_df['Fold'] == fold) & (best_params_df['Dataset'] == 'Combined'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train XGBoost model
    best_xgb = XGBRegressor(random_state=100, **best_params_dict)
    best_xgb.fit(X_train, y_train)

    # Loop through Cooling and Heating datasets
    for dataset_type in ['cooling', 'heating']:
        # Load test data
        test_data = pd.read_csv(f'fold_42_{fold}_filtered_test_data_{dataset_type}.csv')

        # Split features and target
        X_test = test_data.drop(columns=['WH_RTU_Total'])
        y_test = test_data['WH_RTU_Total']

        # Predict and calculate residuals
        y_pred = best_xgb.predict(X_test)
        test_data['Predicted'] = y_pred
        test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

        # Select and rename required columns
        if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
            selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'label']].rename(columns={
                'WH_RTU_Total': 'Actual'
            })
        else:
            raise ValueError(f"'T_out' or 'RH_out' is missing in test data for fold {fold} and dataset {dataset_type}")

        # Add fold, seed, and model columns
        selected_data['Fold'] = fold
        selected_data['Seed'] = 42
        selected_data['Model'] = 'XGBoost'
        selected_data['Experiment'] = 1

        # Append to final merged data
        merged_data.append(selected_data)
        print(f"Processed fold {fold}, dataset {dataset_type}.")

# Combine all folds and datasets into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)
# Save the merged data
merged_final_data.to_csv(f'xgboost_42_combined_residual.csv', index=False)

print("All folds processed and merged data saved successfully.")


Processed fold 1, dataset cooling.
Processed fold 1, dataset heating.
Processed fold 2, dataset cooling.
Processed fold 2, dataset heating.
Processed fold 3, dataset cooling.
Processed fold 3, dataset heating.
Processed fold 4, dataset cooling.
Processed fold 4, dataset heating.
Processed fold 5, dataset cooling.
Processed fold 5, dataset heating.
All folds processed and merged data saved successfully.


## Stacking Regressor

### Stacking Regressor Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

# Placeholder for storing results
results_stacking = []


# Define hyperparameter grid for Stacking Regressor
param_grid_stacking = {
    'final_estimator__fit_intercept': [True, False],  # Meta-model parameter (LinearRegression)
    'rf__n_estimators': [50, 100],                   # Base model: Random Forest
    'rf__max_depth': [10, 20],                       # Base model: Random Forest
    'gb__learning_rate': [0.01, 0.1],                # Base model: Gradient Boosting
    'gb__max_depth': [3, 6]                          # Base model: Gradient Boosting
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold_42_{i}_filtered_train_data.csv')
    test_data_combined = pd.read_csv(f'fold_42_{i}_filtered_test_data.csv')
    test_data_cooling = pd.read_csv(f'fold_42_{i}_filtered_test_data_cooling.csv')
    test_data_heating = pd.read_csv(f'fold_42_{i}_filtered_test_data_heating.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Define base models and Stacking Regressor
    base_models = [
        ('rf', RandomForestRegressor(random_state=100)),
        ('gb', GradientBoostingRegressor(random_state=100))
    ]
    meta_model = LinearRegression()
    stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Perform GridSearchCV on the Stacking Regressor
    grid_search_stacking = GridSearchCV(stacking, param_grid_stacking, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_stacking.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_stacking = grid_search_stacking.best_estimator_
    best_params = grid_search_stacking.best_params_

    # Evaluate on Combined, Cooling, and Heating datasets
    for dataset_type, test_data in [('Combined', test_data_combined),
                                    ('Cooling', test_data_cooling),
                                    ('Heating', test_data_heating)]:
        # Split test features and target
        X_test = test_data.drop(columns=['WH_RTU_Total'])
        y_test = test_data['WH_RTU_Total']

        # Predict and calculate metrics
        y_pred = best_stacking.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results_stacking.append({
            'Fold': i,
            'Seed': 42,  # Add seed column
            'Model': 'Stacking Regressor',
            'Dataset': dataset_type,
            'MAE': mae,
            'R²': r2,
            'Best Params': best_params,
            'Experiment': 1  # Add model column
        })

        print(f"Fold {i}, Model Stacking Regressor, Dataset {dataset_type}, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_stacking_df = pd.DataFrame(results_stacking)

# Save detailed results to CSV
results_stacking_df.to_csv('stacking_42_combined_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for Stacking Regressor:")
summary_stacking = results_stacking_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_stacking)

# Save summary to CSV
summary_stacking.to_csv('stacking_42_combined_hyperparameter_results_summary.csv', index=False)


Fold 1, Model Stacking Regressor, Dataset Combined, MAE: 3.0183, R²: 0.8374, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 3, 'rf__max_depth': 20, 'rf__n_estimators': 50}
Fold 1, Model Stacking Regressor, Dataset Cooling, MAE: 2.7828, R²: 0.7576, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 3, 'rf__max_depth': 20, 'rf__n_estimators': 50}
Fold 1, Model Stacking Regressor, Dataset Heating, MAE: 3.2775, R²: 0.4981, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 3, 'rf__max_depth': 20, 'rf__n_estimators': 50}
Fold 2, Model Stacking Regressor, Dataset Combined, MAE: 2.7837, R²: 0.8492, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 20, 'rf__n_estimators': 50}
Fold 2, Model Stacking Regressor, Dataset Cooling, MAE: 2.3773, R²: 0.7574, Best Params: {'final_estimator__fi

### feature importance for combine stacking regressor

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

# Placeholder for feature importance results
feature_importance_results = []



# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold_42_{fold}_filtered_train_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Extract the best parameters from the results file
    results_stacking_df = pd.read_csv(f'stacking_42_combined_hyperparameter_results_detailed.csv')
    best_params = results_stacking_df.loc[
        (results_stacking_df['Fold'] == fold) & (results_stacking_df['Dataset'] == 'Combined'),
        'Best Params'
    ].iloc[0]
    best_params_dict = eval(best_params)  # Parse string into dictionary

    # Define base models
    base_models = [
        ('rf', RandomForestRegressor(random_state=100, **{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('rf__')})),
        ('gb', GradientBoostingRegressor(random_state=100, **{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('gb__')}))
    ]
    meta_model = LinearRegression(**{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('final_estimator__')})

    # Define Stacking Regressor and fit the model
    stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)
    stacking.fit(X_train, y_train)

    # Calculate feature importances from base models
    for model_name, model in stacking.named_estimators_.items():
        if hasattr(model, 'feature_importances_'):  # Check if the model supports feature importance
            feature_importances = model.feature_importances_
            feature_names = X_train.columns
            importance_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': feature_importances,
                'Model': model_name,
                'Fold': fold,
                'Seed': 42,
                'Experiment': 1  # Add seed column for reference
            })
            feature_importance_results.append(importance_df)

# Combine all feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)

# Save the feature importance results to a file
feature_importance_combined.to_csv(f'stacking_42_combined_feature_importance.csv', index=False)

print(f"Feature importance for seed 42 has been successfully saved.")


Feature importance for seed 42 has been successfully saved.


### generate residual ，predict，label，t_out and RH_OUT

In [ ]:
import pandas as pd
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression



merged_data = []  # Placeholder for storing final merged data

# Load the best model parameters for each fold
best_params_df = pd.read_csv('stacking_42_combined_hyperparameter_results_detailed.csv')

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold_42_{fold}_filtered_train_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Get the best parameters for the current fold
    best_params = best_params_df.loc[best_params_df['Fold'] == fold, 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Define base models and Stacking Regressor with best parameters
    base_models = [
        ('rf', RandomForestRegressor(random_state=100,
                                     n_estimators=best_params_dict['rf__n_estimators'],
                                     max_depth=best_params_dict['rf__max_depth'])),
        ('gb', GradientBoostingRegressor(random_state=100,
                                         learning_rate=best_params_dict['gb__learning_rate'],
                                         max_depth=best_params_dict['gb__max_depth']))
    ]
    meta_model = LinearRegression(fit_intercept=best_params_dict['final_estimator__fit_intercept'])
    best_stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Train the Stacking Regressor
    best_stacking.fit(X_train, y_train)

    # Loop through Cooling and Heating datasets
    for dataset_type in ['cooling', 'heating']:
        # Load test data
        test_data = pd.read_csv(f'fold_42_{fold}_filtered_test_data_{dataset_type}.csv')

        # Split features and target
        X_test = test_data.drop(columns=['WH_RTU_Total'])
        y_test = test_data['WH_RTU_Total']

        # Predict and calculate residuals
        y_pred = best_stacking.predict(X_test)
        test_data['Predicted'] = y_pred
        test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

        # Select and rename required columns
        if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
            selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'label']].rename(columns={
                'WH_RTU_Total': 'Actual'
            })
        else:
            raise ValueError(f"'T_out' or 'RH_out' is missing in test data for fold {fold} and dataset {dataset_type}")

        # Add fold, seed, and model columns
        selected_data['Fold'] = fold
        selected_data['Seed'] = 42
        selected_data['Model'] = 'StackingRegressor'
        selected_data['Experiment'] = 1

        # Append to final merged data
        merged_data.append(selected_data)
        print(f"Processed fold {fold}, dataset {dataset_type}.")

# Combine all folds and datasets into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)
# Save the merged data
merged_final_data.to_csv(f'stacking_42_combined_residual.csv', index=False)

print("All folds processed and merged data saved successfully.")


Processed fold 1, dataset cooling.
Processed fold 1, dataset heating.
Processed fold 2, dataset cooling.
Processed fold 2, dataset heating.
Processed fold 3, dataset cooling.
Processed fold 3, dataset heating.
Processed fold 4, dataset cooling.
Processed fold 4, dataset heating.
Processed fold 5, dataset cooling.
Processed fold 5, dataset heating.
All folds processed and merged data saved successfully.


# Cooling Experiment

do the same model performance for cooling data, first divide the cooling data into 5 folds and generate differet training and testing groups

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

# Load the combined data
cooling_data_with_interactions_42 = pd.read_csv('cooling_data_with_interactions_42.csv')

# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create storage for training and test data splits
folds = []

# Split the data into 5 folds
for train_index, test_index in kf.split(cooling_data_with_interactions_42):
    train_data = cooling_data_with_interactions_42.iloc[train_index]
    test_data = cooling_data_with_interactions_42.iloc[test_index]
    folds.append((train_data, test_data))

# Save each training and test group as separate files
for i, (train_data, test_data) in enumerate(folds):
    train_data.to_csv(f'cooling_train_42_fold{i+1}.csv', index=False)
    test_data.to_csv(f'cooling_test_42_fold{i+1}.csv', index=False)

print("Training and test data for 5 folds with seed 42 have been generated and saved.")


Training and test data for 5 folds with seed 42 have been generated and saved.


do the feature selection solely based on the training data

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Function to perform feature selection
def feature_selection(train_data, target_column, preselected_features, k=25):
    # Separate features and target
    X = train_data.drop(columns=[target_column, 'TIMESTAMP'], errors='ignore')  # Drop TIMESTAMP
    y = train_data[target_column]

    # Train a RandomForestRegressor model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=100)
    rf_model.fit(X, y)

    # Get feature names and their importance scores
    feature_importances = rf_model.feature_importances_
    feature_names = X.columns

    # Combine feature names and scores into a DataFrame
    selected_features = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)

    # Select the preselected features (label, T_out, RH_out)
    preselected_df = selected_features[selected_features['Feature'].isin(preselected_features)]

    # Select the remaining top features after excluding preselected ones
    remaining_features = selected_features[~selected_features['Feature'].isin(preselected_features)]
    top_remaining_features = remaining_features.head(k - len(preselected_features))

    # Combine preselected features with the top remaining features
    final_features = pd.concat([preselected_df, top_remaining_features])

    return final_features

# Perform feature selection for each fold
target_column = 'WH_RTU_Total'
preselected_features = ['label', 'T_out', 'RH_out']

for i in range(1, 6):
    train_data = pd.read_csv(f'cooling_train_42_fold{i}.csv')

    # Perform feature selection
    top_features = feature_selection(train_data, target_column, preselected_features, k=25)

    # Save top features
    top_features.to_csv(f'cooling_train_42_fold{i}_top25_features.csv', index=False)

    print(f"Top 25 features for cooling fold {i} with seed 42 saved.")


Top 25 features for cooling fold 1 with seed 42 saved.
Top 25 features for cooling fold 2 with seed 42 saved.
Top 25 features for cooling fold 3 with seed 42 saved.
Top 25 features for cooling fold 4 with seed 42 saved.
Top 25 features for cooling fold 5 with seed 42 saved.


prepare the cooling dataset's training and test data

In [ ]:
import pandas as pd

# Target variable
target_column = 'WH_RTU_Total'

for i in range(1, 6):
    # Load the original training and test datasets for the current fold
    train_data = pd.read_csv(f'cooling_train_42_fold{i}.csv')
    test_data = pd.read_csv(f'cooling_test_42_fold{i}.csv')

    # Load the top 25 features selected from the current fold
    top_features = pd.read_csv(f'cooling_train_42_fold{i}_top25_features.csv')['Feature'].tolist()

    # Ensure the target column is included in the selected features
    filtered_columns = top_features + [target_column]  # Train data does not include 'label'

    # Filter the training data
    filtered_train_data = train_data[filtered_columns]
    filtered_train_data.to_csv(f'fold{i}_cooling_filtered_train_42_data.csv', index=False)
    print(f"Filtered cooling train data for fold {i} saved.")

    # Filter the test data (temporarily including 'label')
    filtered_test_data = test_data[filtered_columns]

    # Save all datasets
    filtered_test_data.to_csv(f'fold{i}_cooling_filtered_test_42_data.csv', index=False)
    print(f"Filtered cooling test data for fold {i} saved.")



Filtered cooling train data for fold 1 saved.
Filtered cooling test data for fold 1 saved.
Filtered cooling train data for fold 2 saved.
Filtered cooling test data for fold 2 saved.
Filtered cooling train data for fold 3 saved.
Filtered cooling test data for fold 3 saved.
Filtered cooling train data for fold 4 saved.
Filtered cooling test data for fold 4 saved.
Filtered cooling train data for fold 5 saved.
Filtered cooling test data for fold 5 saved.


generate the mae and R2 value for the four models:linear regression, random forest ,xgboost and stacking ensemble regressor.

In [ ]:
!pip install pandas numpy scikit-learn xgboost
!pip install scikit-learn==1.0.2
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
# Placeholder for storing results
results = []

# Loop through each fold
for i in range(1, 6):
    # Load filtered training and test datasets for cooling data
    train_data = pd.read_csv(f'fold{i}_cooling_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_cooling_filtered_test_42_data.csv')

    # Separate features and target for training and testing data
    target_column = 'WH_RTU_Total'
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column])
    y_test = test_data[target_column]

    # Define models with default parameters
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(random_state=100),
        'XGBoost': XGBRegressor(random_state=100),
        'Stacking Regressor': StackingRegressor(
            estimators=[
                ('lr', LinearRegression()),
                ('rf', RandomForestRegressor(random_state=100)),
                ('xgb', XGBRegressor(random_state=100))
            ]
        )
    }

    # Train and evaluate each model
    for model_name, model in models.items():
        # Train model
        model.fit(X_train, y_train)

        # Predict on test data
        y_pred = model.predict(X_test)

        # Calculate MAE and R²
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results.append({
            'Fold': i,
            'Seed': 42,
            'Model': model_name,
            'Dataset': 'Cooling',
            'MAE': mae,
            'R²': r2,
            'Experiment':2
        })

        print(f"Fold {i}, Model {model_name}, MAE: {mae:.4f}, R²: {r2:.4f}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv('cooling_data_results_mae_r2.csv', index=False)

# Display overall summary
print("\nSummary Results for All Models (Cooling Data):")
summary = results_df.groupby('Model').agg({'MAE': 'mean', 'R²': 'mean'}).sort_values(by='MAE')
print(summary)


Fold 1, Model Linear Regression, MAE: 5.1169, R²: 0.4179
Fold 1, Model Random Forest, MAE: 2.7831, R²: 0.6805
Fold 1, Model XGBoost, MAE: 2.6708, R²: 0.7021
Fold 1, Model Stacking Regressor, MAE: 2.6855, R²: 0.7085
Fold 2, Model Linear Regression, MAE: 4.6243, R²: 0.5093
Fold 2, Model Random Forest, MAE: 2.1688, R²: 0.7803
Fold 2, Model XGBoost, MAE: 2.2281, R²: 0.7741
Fold 2, Model Stacking Regressor, MAE: 2.1163, R²: 0.7900
Fold 3, Model Linear Regression, MAE: 4.7232, R²: 0.5346
Fold 3, Model Random Forest, MAE: 2.1348, R²: 0.8201
Fold 3, Model XGBoost, MAE: 2.1085, R²: 0.8335
Fold 3, Model Stacking Regressor, MAE: 2.0778, R²: 0.8331
Fold 4, Model Linear Regression, MAE: 4.7064, R²: 0.4764
Fold 4, Model Random Forest, MAE: 2.2271, R²: 0.7792
Fold 4, Model XGBoost, MAE: 2.3552, R²: 0.7620
Fold 4, Model Stacking Regressor, MAE: 2.2399, R²: 0.7819
Fold 5, Model Linear Regression, MAE: 4.7443, R²: 0.4749
Fold 5, Model Random Forest, MAE: 2.1909, R²: 0.8116
Fold 5, Model XGBoost, MAE: 2.

## Random Forest

### hyper parameter tuning for cooling random forest

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_rf = []

# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold{i}_cooling_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_cooling_filtered_test_42_data.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Initialize Random Forest and perform GridSearchCV
    rf = RandomForestRegressor(random_state=100)
    grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_rf.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_rf = grid_search_rf.best_estimator_
    best_params = grid_search_rf.best_params_

    # Split test features and target
    X_test = test_data.drop(columns=['WH_RTU_Total'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate metrics
    y_pred = best_rf.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results_rf.append({
        'Fold': i,
        'Seed':42,
        'Model': 'Random Forest',
        'Dataset': 'Cooling',
        'MAE': mae,
        'R²': r2,
        'Best Params': best_params,
        'Experiment': 2
    })

    print(f"Fold {i}, Model Random Forest, Dataset Cooling, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_rf_df = pd.DataFrame(results_rf)

# Save detailed results to CSV
results_rf_df.to_csv('random_forest_42_cooling_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for Random Forest:")
summary_rf = results_rf_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_rf)

# Save summary to CSV
summary_rf.to_csv('random_forest_42_cooling_summary_results.csv', index=False)


Fold 1, Model Random Forest, Dataset Cooling, MAE: 2.7388, R²: 0.6882, Best Params: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 150}
Fold 2, Model Random Forest, Dataset Cooling, MAE: 2.1836, R²: 0.7787, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}
Fold 3, Model Random Forest, Dataset Cooling, MAE: 2.1184, R²: 0.8191, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}
Fold 4, Model Random Forest, Dataset Cooling, MAE: 2.2043, R²: 0.7800, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Fold 5, Model Random Forest, Dataset Cooling, MAE: 2.1781, R²: 0.8090, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}

Summary Results for Random Forest:
           Model  Dataset       MAE        R²
0  Random Forest  Cooling  2.284662  0.775014


### Feature Importance for Random Forest

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Placeholder for feature importance results
feature_importance_results = []

# Fixed seed value

experiment = 2  # Experiment number

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_cooling_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    results_rf_df = pd.read_csv('random_forest_42_cooling_hyperparameter_results_detailed.csv')
    best_params = results_rf_df.loc[(results_rf_df['Fold'] == fold) & (results_rf_df['Dataset'] == 'Cooling'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train Random Forest model with best parameters
    best_rf = RandomForestRegressor(random_state=100, **best_params_dict)
    best_rf.fit(X_train, y_train)

    # Calculate feature importance
    feature_importances = best_rf.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances,
        'Fold': fold,
        'Seed': 42,  # Add seed column
        'Model': 'Random Forest',
        'Experiment': experiment  # Add experiment column
    })
    feature_importance_results.append(importance_df)

# Combine and save feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)
feature_importance_combined.to_csv(f'random_forest_42_cooling_feature_importance_experiment_{experiment}.csv', index=False)

print(f"Feature importance for seed 42 and experiment {experiment} has been successfully saved.")

Feature importance for seed 42 and experiment 2 has been successfully saved.


### residual，predict，label，T_OUT and RH_OUT

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


experiment = 2  # Experiment number
merged_data = []  # Placeholder for storing final merged data

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_cooling_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    best_params_df = pd.read_csv('random_forest_42_cooling_hyperparameter_results_detailed.csv')
    best_params = best_params_df.loc[(best_params_df['Fold'] == fold) & (best_params_df['Dataset'] == 'Cooling'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train Random Forest model with best parameters
    best_rf = RandomForestRegressor(random_state=100, **best_params_dict)
    best_rf.fit(X_train, y_train)

    # Load test data
    test_data = pd.read_csv(f'fold{fold}_cooling_filtered_test_42_data.csv')
    test_data['Label'] = 'Cooling'  # Dynamically add label based on dataset type

    # Split features and target
    X_test = test_data.drop(columns=['WH_RTU_Total', 'Label'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate residuals
    y_pred = best_rf.predict(X_test)
    test_data['Predicted'] = y_pred
    test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

    # Select and rename required columns
    if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
        selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'Label']].rename(columns={
            'WH_RTU_Total': 'Actual'
        })
    else:
        missing_columns = [col for col in ['T_out', 'RH_out'] if col not in test_data.columns]
        raise ValueError(f"{', '.join(missing_columns)} is/are missing in test data for fold {fold}")

    # Add fold, seed, and model columns
    selected_data['Fold'] = fold
    selected_data['Seed'] = 42
    selected_data['Model'] = 'Random Forest'
    selected_data['Experiment'] = experiment

    # Append to final merged data
    merged_data.append(selected_data)
    print(f"Processed fold {fold}.")

# Combine all folds into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)

# Save the merged data
output_file = 'random_forest_42_cooling_residual.csv'
merged_final_data.to_csv(output_file, index=False)

print(f"All folds processed and merged data saved successfully to {output_file}.")


Processed fold 1.
Processed fold 2.
Processed fold 3.
Processed fold 4.
Processed fold 5.
All folds processed and merged data saved successfully to random_forest_42_cooling_residual.csv.


## XGBOOST

### hyper parameter tuning for cooling XGBoost

In [ ]:
!pip install pandas numpy scikit-learn xgboost
!pip install scikit-learn==1.0.2
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_xgb = []
experiment = 2  # Experiment number
# Define hyperparameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold{i}_cooling_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_cooling_filtered_test_42_data.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Initialize XGBoost and perform GridSearchCV
    xgb = XGBRegressor(random_state=100)
    grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_xgb.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_xgb = grid_search_xgb.best_estimator_
    best_params = grid_search_xgb.best_params_

    # Split test features and target
    X_test = test_data.drop(columns=['WH_RTU_Total'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate metrics
    y_pred = best_xgb.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results_xgb.append({
        'Fold': i,
        'Seed': 42,
        'Model': 'XGBoost',
        'Dataset': 'Cooling',
        'MAE': mae,
        'R²': r2,
        'Best Params': best_params,
        'Experiment' : experiment
    })


    print(f"Fold {i}, Model XGBoost, Dataset Cooling, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_xgb_df = pd.DataFrame(results_xgb)

# Save detailed results to CSV
results_xgb_df.to_csv('xgboost_42_cooling_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for XGBoost:")
summary_xgb = results_xgb_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_xgb)

# Save summary to CSV
summary_xgb.to_csv('xgboost_42_cooling_summary_results.csv', index=False)


Fold 1, Model XGBoost, Dataset Cooling, MAE: 2.5667, R²: 0.7111, Best Params: {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 150, 'subsample': 0.8}
Fold 2, Model XGBoost, Dataset Cooling, MAE: 2.1399, R²: 0.7921, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 3, Model XGBoost, Dataset Cooling, MAE: 1.9866, R²: 0.8425, Best Params: {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 150, 'subsample': 0.8}
Fold 4, Model XGBoost, Dataset Cooling, MAE: 2.1882, R²: 0.7896, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 5, Model XGBoost, Dataset Cooling, MAE: 2.0958, R²: 0.8193, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}

Summary Results for XGBoost:
     Model  Dataset       MAE      R²
0  XGBoost  Cooling  2.195429  0.7909


### Feature Importance

In [ ]:
import pandas as pd
from xgboost import XGBRegressor

# Placeholder for feature importance results
feature_importance_results = []


experiment = 2  # Experiment number

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_cooling_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Extract the best parameters from the results file
    results_xgb_df = pd.read_csv('xgboost_42_cooling_hyperparameter_results_detailed.csv')
    best_params = results_xgb_df.loc[
        (results_xgb_df['Fold'] == fold) & (results_xgb_df['Dataset'] == 'Cooling'),
        'Best Params'
    ].iloc[0]
    best_params_dict = eval(best_params)  # Parse string into dictionary

    # Initialize and train the model with the best parameters
    best_xgb = XGBRegressor(random_state=100, **best_params_dict)
    best_xgb.fit(X_train, y_train)

    # Calculate feature importance
    feature_importances = best_xgb.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances,
        'Fold': fold,
        'Seed': 42,  # Add seed column
        'Model': 'XGBoost',
        'Experiment': experiment  # Add experiment column
    })
    feature_importance_results.append(importance_df)

# Combine all feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)

# Save the feature importance results to a file
feature_importance_combined.to_csv(f'xgboost_42_cooling_feature_importance_experiment_{experiment}.csv', index=False)

print(f"Feature importance for seed 42 and experiment {experiment} has been successfully saved.")

Feature importance for seed 42 and experiment 2 has been successfully saved.


### Residual analysis

In [ ]:
import pandas as pd
from xgboost import XGBRegressor


experiment = 2  # Experiment number
merged_data = []  # Placeholder for storing final merged data

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_cooling_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    best_params_df = pd.read_csv('xgboost_42_cooling_hyperparameter_results_detailed.csv')
    best_params = best_params_df.loc[(best_params_df['Fold'] == fold) & (best_params_df['Dataset'] == 'Cooling'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train XGBoost model with best parameters
    best_xgb = XGBRegressor(random_state=100, **best_params_dict)
    best_xgb.fit(X_train, y_train)

    # Load test data
    test_data = pd.read_csv(f'fold{fold}_cooling_filtered_test_42_data.csv')
    test_data['Label'] = 'Cooling'  # Dynamically add label based on dataset type

    # Split features and target
    X_test = test_data.drop(columns=['WH_RTU_Total', 'Label'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate residuals
    y_pred = best_xgb.predict(X_test)
    test_data['Predicted'] = y_pred
    test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

    # Select and rename required columns
    if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
        selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'Label']].rename(columns={
            'WH_RTU_Total': 'Actual'
        })
    else:
        missing_columns = [col for col in ['T_out', 'RH_out'] if col not in test_data.columns]
        raise ValueError(f"{', '.join(missing_columns)} is/are missing in test data for fold {fold}")

    # Add fold, seed, and model columns
    selected_data['Fold'] = fold
    selected_data['Seed'] = 42
    selected_data['Model'] = 'XGBoost'
    selected_data['Experiment'] = experiment

    # Append to final merged data
    merged_data.append(selected_data)
    print(f"Processed fold {fold}.")

# Combine all folds into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)

# Save the merged data
output_file = 'xgboost_42_cooling_residual.csv'
merged_final_data.to_csv(output_file, index=False)

print(f"All folds processed and merged data saved successfully to {output_file}.")


Processed fold 1.
Processed fold 2.
Processed fold 3.
Processed fold 4.
Processed fold 5.
All folds processed and merged data saved successfully to xgboost_42_cooling_residual.csv.


## Stacking Regressor

### hyper parameter tuning for cooling Stacking Regressor

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_stacking = []

experiment = 2  # Experiment number
# Define hyperparameter grid for Stacking Regressor
param_grid_stacking = {
    'final_estimator__fit_intercept': [True, False],  # Meta-model parameter (LinearRegression)
    'rf__n_estimators': [50, 100],                   # Base model: Random Forest
    'rf__max_depth': [10, 20],                       # Base model: Random Forest
    'gb__learning_rate': [0.01, 0.1],                # Base model: Gradient Boosting
    'gb__max_depth': [3, 6]                          # Base model: Gradient Boosting
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold{i}_cooling_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_cooling_filtered_test_42_data.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Define base models and Stacking Regressor
    base_models = [
        ('rf', RandomForestRegressor(random_state=100)),
        ('gb', GradientBoostingRegressor(random_state=100))
    ]
    meta_model = LinearRegression()
    stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Perform GridSearchCV on the Stacking Regressor
    grid_search_stacking = GridSearchCV(stacking, param_grid_stacking, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_stacking.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_stacking = grid_search_stacking.best_estimator_
    best_params = grid_search_stacking.best_params_

    # Split test features and target
    X_test = test_data.drop(columns=['WH_RTU_Total'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate metrics
    y_pred = best_stacking.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results_stacking.append({
        'Fold': i,
        'Seed': 42,
        'Model': 'Stacking Regressor',
        'Dataset': 'Cooling',
        'MAE': mae,
        'R²': r2,
        'Best Params': best_params,
        'Experiment': experiment
    })

    print(f"Fold {i}, Model Stacking Regressor, Dataset Cooling, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_stacking_df = pd.DataFrame(results_stacking)

# Save detailed results to CSV
results_stacking_df.to_csv('stacking_42_cooling_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for Stacking Regressor:")
summary_stacking = results_stacking_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_stacking)

# Save summary to CSV
summary_stacking.to_csv('stacking_42_cooling_summary_results.csv', index=False)


Fold 1, Model Stacking Regressor, Dataset Cooling, MAE: 2.6437, R²: 0.7122, Best Params: {'final_estimator__fit_intercept': True, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 10, 'rf__n_estimators': 50}
Fold 2, Model Stacking Regressor, Dataset Cooling, MAE: 2.1481, R²: 0.7853, Best Params: {'final_estimator__fit_intercept': True, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 10, 'rf__n_estimators': 100}
Fold 3, Model Stacking Regressor, Dataset Cooling, MAE: 2.0945, R²: 0.8251, Best Params: {'final_estimator__fit_intercept': True, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 10, 'rf__n_estimators': 100}
Fold 4, Model Stacking Regressor, Dataset Cooling, MAE: 2.1366, R²: 0.7957, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 20, 'rf__n_estimators': 100}
Fold 5, Model Stacking Regressor, Dataset Cooling, MAE: 2.1276, R²: 0.8115, Best Params: {'final_estimator__fit_

### Feature Importance

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

# Placeholder for feature importance results
feature_importance_results = []


experiment = 2  # Experiment number

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_cooling_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Extract the best parameters from the results file
    results_stacking_df = pd.read_csv('stacking_42_cooling_hyperparameter_results_detailed.csv')
    best_params = results_stacking_df.loc[
        (results_stacking_df['Fold'] == fold) & (results_stacking_df['Dataset'] == 'Cooling'),
        'Best Params'
    ].iloc[0]
    best_params_dict = eval(best_params)  # Parse string into dictionary

    # Define base models
    base_models = [
        ('rf', RandomForestRegressor(random_state=100, **{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('rf__')})),
        ('gb', GradientBoostingRegressor(random_state=100, **{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('gb__')}))
    ]
    meta_model = LinearRegression(**{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('final_estimator__')})

    # Define Stacking Regressor and fit the model
    stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)
    stacking.fit(X_train, y_train)

    # Calculate feature importances from base models
    for model_name, model in stacking.named_estimators_.items():
        if hasattr(model, 'feature_importances_'):  # Check if the model supports feature importance
            feature_importances = model.feature_importances_
            feature_names = X_train.columns
            importance_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': feature_importances,
                'Model': model_name,
                'Fold': fold,
                'Seed': 42,
                'Experiment': experiment  # Add experiment column
            })
            feature_importance_results.append(importance_df)

# Combine all feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)

# Save the feature importance results to a file
feature_importance_combined.to_csv(f'stacking_42_cooling_feature_importance_experiment_{experiment}.csv', index=False)

print(f"Feature importance for seed 42 and experiment {experiment} has been successfully saved.")

Feature importance for seed 42 and experiment 2 has been successfully saved.


### Residual Analysis

In [ ]:
import pandas as pd
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression


experiment = 2  # Experiment number
merged_data = []  # Placeholder for storing final merged data

# Load the best model parameters for each fold
best_params_df = pd.read_csv('stacking_42_cooling_hyperparameter_results_detailed.csv')

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_cooling_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Get the best parameters for the current fold
    best_params = best_params_df.loc[best_params_df['Fold'] == fold, 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Define base models and Stacking Regressor with best parameters
    base_models = [
        ('rf', RandomForestRegressor(random_state=100,
                                     n_estimators=best_params_dict['rf__n_estimators'],
                                     max_depth=best_params_dict['rf__max_depth'])),
        ('gb', GradientBoostingRegressor(random_state=100,
                                         learning_rate=best_params_dict['gb__learning_rate'],
                                         max_depth=best_params_dict['gb__max_depth']))
    ]
    meta_model = LinearRegression(fit_intercept=best_params_dict['final_estimator__fit_intercept'])
    best_stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Train the Stacking Regressor
    best_stacking.fit(X_train, y_train)

    # Load test data
    test_data = pd.read_csv(f'fold{fold}_cooling_filtered_test_42_data.csv')
    test_data['Label'] = 'Cooling'  # Dynamically add label based on dataset type

    # Split features and target
    X_test = test_data.drop(columns=['WH_RTU_Total', 'Label'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate residuals
    y_pred = best_stacking.predict(X_test)
    test_data = test_data.reset_index(drop=True)  # Reset test_data index
    y_pred_series = pd.Series(y_pred, index=test_data.index)  # Align y_pred with test_data index
    test_data['Predicted'] = y_pred_series
    test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

    # Select and rename required columns
    if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
        selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'Label']].rename(columns={
            'WH_RTU_Total': 'Actual'
        })
    else:
        missing_columns = [col for col in ['T_out', 'RH_out'] if col not in test_data.columns]
        raise ValueError(f"{', '.join(missing_columns)} is/are missing in test data for fold {fold}")

    # Add fold, seed, and model columns
    selected_data['Fold'] = fold
    selected_data['Seed'] = 42
    selected_data['Model'] = 'StackingRegressor'
    selected_data['Experiment'] = experiment

    # Append to final merged data
    merged_data.append(selected_data)
    print(f"Processed fold {fold}.")

# Combine all folds into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)

# Save the merged data
output_file = 'stacking_42_cooling_residual.csv'
merged_final_data.to_csv(output_file, index=False)

print(f"All folds processed and merged data saved successfully to {output_file}.")


Processed fold 1.
Processed fold 2.
Processed fold 3.
Processed fold 4.
Processed fold 5.
All folds processed and merged data saved successfully to stacking_42_cooling_residual.csv.


# heating Experiment

do the same model performance for heating data, first divide the heating data into 5 folds and generate differet training and testing groups

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

# Load the combined data
heating_data_with_interactions_42 = pd.read_csv('heating_data_with_interactions_42.csv')

# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create storage for training and test data splits
folds = []

# Split the data into 5 folds
for train_index, test_index in kf.split(heating_data_with_interactions_42):
    train_data = heating_data_with_interactions_42.iloc[train_index]
    test_data = heating_data_with_interactions_42.iloc[test_index]
    folds.append((train_data, test_data))

# Save each training and test group as separate files
for i, (train_data, test_data) in enumerate(folds):
    train_data.to_csv(f'heating_train_42_fold{i+1}.csv', index=False)
    test_data.to_csv(f'heating_test_42_fold{i+1}.csv', index=False)

print("Training and test data for 5 folds with seed 42 have been generated and saved.")


Training and test data for 5 folds with seed 42 have been generated and saved.


do the feature selection solely based on the training data

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Function to perform feature selection
def feature_selection(train_data, target_column, preselected_features, k=25):
    # Separate features and target
    X = train_data.drop(columns=[target_column, 'TIMESTAMP'], errors='ignore')  # Drop TIMESTAMP
    y = train_data[target_column]

    # Train a RandomForestRegressor model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=100)
    rf_model.fit(X, y)

    # Get feature names and their importance scores
    feature_importances = rf_model.feature_importances_
    feature_names = X.columns

    # Combine feature names and scores into a DataFrame
    selected_features = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)

    # Select the preselected features (label, T_out, RH_out)
    preselected_df = selected_features[selected_features['Feature'].isin(preselected_features)]

    # Select the remaining top features after excluding preselected ones
    remaining_features = selected_features[~selected_features['Feature'].isin(preselected_features)]
    top_remaining_features = remaining_features.head(k - len(preselected_features))

    # Combine preselected features with the top remaining features
    final_features = pd.concat([preselected_df, top_remaining_features])

    return final_features

# Perform feature selection for each fold
target_column = 'WH_RTU_Total'
preselected_features = ['label', 'T_out', 'RH_out']

for i in range(1, 6):
    train_data = pd.read_csv(f'heating_train_42_fold{i}.csv')

    # Perform feature selection
    top_features = feature_selection(train_data, target_column, preselected_features, k=25)

    # Save top features
    top_features.to_csv(f'heating_train_42_fold{i}_top25_features.csv', index=False)

    print(f"Top 25 features for heating fold {i} with seed 42 saved.")


KeyboardInterrupt: 

prepare the heating dataset's training and test data

In [ ]:
import pandas as pd

# Target variable
target_column = 'WH_RTU_Total'

for i in range(1, 6):
    # Load the original training and test datasets for the current fold
    train_data = pd.read_csv(f'heating_train_42_fold{i}.csv')
    test_data = pd.read_csv(f'heating_test_42_fold{i}.csv')

    # Load the top 25 features selected from the current fold
    top_features = pd.read_csv(f'heating_train_42_fold{i}_top25_features.csv')['Feature'].tolist()

    # Ensure the target column is included in the selected features
    filtered_columns = top_features + [target_column]  # Train data does not include 'label'

    # Filter the training data
    filtered_train_data = train_data[filtered_columns]
    filtered_train_data.to_csv(f'fold{i}_heating_filtered_train_42_data.csv', index=False)
    print(f"Filtered heating train data for fold {i} saved.")

    # Filter the test data (temporarily including 'label')
    filtered_test_data = test_data[filtered_columns]

    # Save all datasets
    filtered_test_data.to_csv(f'fold{i}_heating_filtered_test_42_data.csv', index=False)
    print(f"Filtered heating test data for fold {i} saved.")



Filtered heating train data for fold 1 saved.
Filtered heating test data for fold 1 saved.
Filtered heating train data for fold 2 saved.
Filtered heating test data for fold 2 saved.
Filtered heating train data for fold 3 saved.
Filtered heating test data for fold 3 saved.
Filtered heating train data for fold 4 saved.
Filtered heating test data for fold 4 saved.
Filtered heating train data for fold 5 saved.
Filtered heating test data for fold 5 saved.


generate the mae and R2 value for the four models:linear regression, random forest ,xgboost and stacking ensemble regressor.

In [ ]:
!pip install pandas numpy scikit-learn xgboost
!pip install scikit-learn==1.0.2
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
# Placeholder for storing results
results = []

# Loop through each fold
for i in range(1, 6):
    # Load filtered training and test datasets for heating data
    train_data = pd.read_csv(f'fold{i}_heating_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_heating_filtered_test_42_data.csv')

    # Separate features and target for training and testing data
    target_column = 'WH_RTU_Total'
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column])
    y_test = test_data[target_column]

    # Define models with default parameters
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(random_state=100),
        'XGBoost': XGBRegressor(random_state=100),
        'Stacking Regressor': StackingRegressor(
            estimators=[
                ('lr', LinearRegression()),
                ('rf', RandomForestRegressor(random_state=100)),
                ('xgb', XGBRegressor(random_state=100))
            ]
        )
    }

    # Train and evaluate each model
    for model_name, model in models.items():
        # Train model
        model.fit(X_train, y_train)

        # Predict on test data
        y_pred = model.predict(X_test)

        # Calculate MAE and R²
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results.append({
            'Fold': i,
            'Seed': 42,
            'Model': model_name,
            'Dataset': 'heating',
            'MAE': mae,
            'R²': r2,
            'Experiment':2
        })

        print(f"Fold {i}, Model {model_name}, MAE: {mae:.4f}, R²: {r2:.4f}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv('heating_data_results_mae_r2.csv', index=False)

# Display overall summary
print("\nSummary Results for All Models (heating Data):")
summary = results_df.groupby('Model').agg({'MAE': 'mean', 'R²': 'mean'}).sort_values(by='MAE')
print(summary)


Fold 1, Model Linear Regression, MAE: 3.5158, R²: 0.3482
Fold 1, Model Random Forest, MAE: 2.8247, R²: 0.5646
Fold 1, Model XGBoost, MAE: 2.8222, R²: 0.5362
Fold 1, Model Stacking Regressor, MAE: 2.9881, R²: 0.5380
Fold 2, Model Linear Regression, MAE: 3.5004, R²: 0.3841
Fold 2, Model Random Forest, MAE: 2.9498, R²: 0.5709
Fold 2, Model XGBoost, MAE: 2.7744, R²: 0.5668
Fold 2, Model Stacking Regressor, MAE: 3.0451, R²: 0.5506
Fold 3, Model Linear Regression, MAE: 3.7288, R²: 0.3719
Fold 3, Model Random Forest, MAE: 3.2049, R²: 0.5496
Fold 3, Model XGBoost, MAE: 3.2185, R²: 0.5081
Fold 3, Model Stacking Regressor, MAE: 3.2651, R²: 0.5450
Fold 4, Model Linear Regression, MAE: 3.6318, R²: 0.4350
Fold 4, Model Random Forest, MAE: 2.9965, R²: 0.6144
Fold 4, Model XGBoost, MAE: 2.9355, R²: 0.6128
Fold 4, Model Stacking Regressor, MAE: 3.0255, R²: 0.6268
Fold 5, Model Linear Regression, MAE: 3.5106, R²: 0.4494
Fold 5, Model Random Forest, MAE: 2.8340, R²: 0.6148
Fold 5, Model XGBoost, MAE: 2.

## Random Forest

### hyper parameter tuning for heating random forest

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_rf = []

# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold{i}_heating_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_heating_filtered_test_42_data.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Initialize Random Forest and perform GridSearchCV
    rf = RandomForestRegressor(random_state=100)
    grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_rf.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_rf = grid_search_rf.best_estimator_
    best_params = grid_search_rf.best_params_

    # Split test features and target
    X_test = test_data.drop(columns=['WH_RTU_Total'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate metrics
    y_pred = best_rf.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results_rf.append({
        'Fold': i,
        'Seed':42,
        'Model': 'Random Forest',
        'Dataset': 'heating',
        'MAE': mae,
        'R²': r2,
        'Best Params': best_params,
        'Experiment': 2
    })

    print(f"Fold {i}, Model Random Forest, Dataset heating, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_rf_df = pd.DataFrame(results_rf)

# Save detailed results to CSV
results_rf_df.to_csv('random_forest_42_heating_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for Random Forest:")
summary_rf = results_rf_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_rf)

# Save summary to CSV
summary_rf.to_csv('random_forest_42_heating_summary_results.csv', index=False)


Fold 1, Model Random Forest, Dataset heating, MAE: 2.8347, R²: 0.5615, Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 150}
Fold 2, Model Random Forest, Dataset heating, MAE: 2.9154, R²: 0.5796, Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 150}
Fold 3, Model Random Forest, Dataset heating, MAE: 3.2533, R²: 0.5425, Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}
Fold 4, Model Random Forest, Dataset heating, MAE: 3.0317, R²: 0.6062, Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Fold 5, Model Random Forest, Dataset heating, MAE: 2.8340, R²: 0.6148, Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Summary Results for Random Forest:
           Model  Dataset       MAE       R²
0  Random Forest  heating  2.973816  0.58092


### Feature Importance for Random Forest

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Placeholder for feature importance results
feature_importance_results = []

# Fixed seed value

experiment = 2  # Experiment number

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_heating_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    results_rf_df = pd.read_csv('random_forest_42_heating_hyperparameter_results_detailed.csv')
    best_params = results_rf_df.loc[(results_rf_df['Fold'] == fold) & (results_rf_df['Dataset'] == 'heating'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train Random Forest model with best parameters
    best_rf = RandomForestRegressor(random_state=100, **best_params_dict)
    best_rf.fit(X_train, y_train)

    # Calculate feature importance
    feature_importances = best_rf.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances,
        'Fold': fold,
        'Seed': 42,  # Add seed column
        'Model': 'Random Forest',
        'Experiment': experiment  # Add experiment column
    })
    feature_importance_results.append(importance_df)

# Combine and save feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)
feature_importance_combined.to_csv(f'random_forest_42_heating_feature_importance_experiment_{experiment}.csv', index=False)

print(f"Feature importance for seed 42 and experiment {experiment} has been successfully saved.")

Feature importance for seed 42 and experiment 2 has been successfully saved.


### residual，predict，label，T_OUT and RH_OUT

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Fixed seed value

experiment = 2  # Experiment number
merged_data = []  # Placeholder for storing final merged data

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_heating_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    best_params_df = pd.read_csv('random_forest_42_heating_hyperparameter_results_detailed.csv')
    best_params = best_params_df.loc[(best_params_df['Fold'] == fold) & (best_params_df['Dataset'] == 'heating'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train Random Forest model with best parameters
    best_rf = RandomForestRegressor(random_state=100, **best_params_dict)
    best_rf.fit(X_train, y_train)

    # Load test data
    test_data = pd.read_csv(f'fold{fold}_heating_filtered_test_42_data.csv')
    test_data['Label'] = 'heating'  # Dynamically add label based on dataset type

    # Split features and target
    X_test = test_data.drop(columns=['WH_RTU_Total', 'Label'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate residuals
    y_pred = best_rf.predict(X_test)
    test_data['Predicted'] = y_pred
    test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

    # Select and rename required columns
    if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
        selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'Label']].rename(columns={
            'WH_RTU_Total': 'Actual'
        })
    else:
        missing_columns = [col for col in ['T_out', 'RH_out'] if col not in test_data.columns]
        raise ValueError(f"{', '.join(missing_columns)} is/are missing in test data for fold {fold}")

    # Add fold, seed, and model columns
    selected_data['Fold'] = fold
    selected_data['Seed'] = 42
    selected_data['Model'] = 'Random Forest'
    selected_data['Experiment'] = experiment

    # Append to final merged data
    merged_data.append(selected_data)
    print(f"Processed fold {fold}.")

# Combine all folds into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)

# Save the merged data
output_file = 'random_forest_42_heating_residual.csv'
merged_final_data.to_csv(output_file, index=False)

print(f"All folds processed and merged data saved successfully to {output_file}.")


Processed fold 1.
Processed fold 2.
Processed fold 3.
Processed fold 4.
Processed fold 5.
All folds processed and merged data saved successfully to random_forest_42_heating_residual.csv.


## XGBOOST

### hyper parameter tuning for heating XGBoost

In [ ]:
!pip install pandas numpy scikit-learn xgboost
!pip install scikit-learn==1.0.2
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_xgb = []
experiment = 2  # Experiment number
# Define hyperparameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold{i}_heating_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_heating_filtered_test_42_data.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Initialize XGBoost and perform GridSearchCV
    xgb = XGBRegressor(random_state=100)
    grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_xgb.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_xgb = grid_search_xgb.best_estimator_
    best_params = grid_search_xgb.best_params_

    # Split test features and target
    X_test = test_data.drop(columns=['WH_RTU_Total'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate metrics
    y_pred = best_xgb.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results_xgb.append({
        'Fold': i,
        'Seed': 42,
        'Model': 'XGBoost',
        'Dataset': 'heating',
        'MAE': mae,
        'R²': r2,
        'Best Params': best_params,
        'Experiment' : experiment
    })


    print(f"Fold {i}, Model XGBoost, Dataset heating, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_xgb_df = pd.DataFrame(results_xgb)

# Save detailed results to CSV
results_xgb_df.to_csv('xgboost_42_heating_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for XGBoost:")
summary_xgb = results_xgb_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_xgb)

# Save summary to CSV
summary_xgb.to_csv('xgboost_42_heating_summary_results.csv', index=False)


Fold 1, Model XGBoost, Dataset heating, MAE: 2.7617, R²: 0.5551, Best Params: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 2, Model XGBoost, Dataset heating, MAE: 2.7254, R²: 0.6055, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 3, Model XGBoost, Dataset heating, MAE: 3.2929, R²: 0.5191, Best Params: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Fold 4, Model XGBoost, Dataset heating, MAE: 2.9266, R²: 0.6234, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Fold 5, Model XGBoost, Dataset heating, MAE: 2.7558, R²: 0.6267, Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}

Summary Results for XGBoost:
     Model  Dataset       MAE        R²
0  XGBoost  heating  2.892474  0.585945


### Feature Importance

In [ ]:
import pandas as pd
from xgboost import XGBRegressor

# Placeholder for feature importance results
feature_importance_results = []

# Fixed random seed

experiment = 2  # Experiment number

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_heating_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Extract the best parameters from the results file
    results_xgb_df = pd.read_csv('xgboost_42_heating_hyperparameter_results_detailed.csv')
    best_params = results_xgb_df.loc[
        (results_xgb_df['Fold'] == fold) & (results_xgb_df['Dataset'] == 'heating'),
        'Best Params'
    ].iloc[0]
    best_params_dict = eval(best_params)  # Parse string into dictionary

    # Initialize and train the model with the best parameters
    best_xgb = XGBRegressor(random_state=100, **best_params_dict)
    best_xgb.fit(X_train, y_train)

    # Calculate feature importance
    feature_importances = best_xgb.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances,
        'Fold': fold,
        'Seed': 42,  # Add seed column
        'Model': 'XGBoost',
        'Experiment': experiment  # Add experiment column
    })
    feature_importance_results.append(importance_df)

# Combine all feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)

# Save the feature importance results to a file
feature_importance_combined.to_csv(f'xgboost_42_heating_feature_importance_experiment_{experiment}.csv', index=False)

print(f"Feature importance for seed 42 and experiment {experiment} has been successfully saved.")

Feature importance for seed 42 and experiment 2 has been successfully saved.


### Residual analysis

In [ ]:
import pandas as pd
from xgboost import XGBRegressor


experiment = 2  # Experiment number
merged_data = []  # Placeholder for storing final merged data

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_heating_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Load the best model parameters for the current fold
    best_params_df = pd.read_csv('xgboost_42_heating_hyperparameter_results_detailed.csv')
    best_params = best_params_df.loc[(best_params_df['Fold'] == fold) & (best_params_df['Dataset'] == 'heating'), 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Train XGBoost model with best parameters
    best_xgb = XGBRegressor(random_state=100, **best_params_dict)
    best_xgb.fit(X_train, y_train)

    # Load test data
    test_data = pd.read_csv(f'fold{fold}_heating_filtered_test_42_data.csv')
    test_data['Label'] = 'heating'  # Dynamically add label based on dataset type

    # Split features and target
    X_test = test_data.drop(columns=['WH_RTU_Total', 'Label'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate residuals
    y_pred = best_xgb.predict(X_test)
    test_data['Predicted'] = y_pred
    test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

    # Select and rename required columns
    if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
        selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'Label']].rename(columns={
            'WH_RTU_Total': 'Actual'
        })
    else:
        missing_columns = [col for col in ['T_out', 'RH_out'] if col not in test_data.columns]
        raise ValueError(f"{', '.join(missing_columns)} is/are missing in test data for fold {fold}")

    # Add fold, seed, and model columns
    selected_data['Fold'] = fold
    selected_data['Seed'] = 42
    selected_data['Model'] = 'XGBoost'
    selected_data['Experiment'] = experiment

    # Append to final merged data
    merged_data.append(selected_data)
    print(f"Processed fold {fold}.")

# Combine all folds into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)

# Save the merged data
output_file = 'xgboost_42_heating_residual.csv'
merged_final_data.to_csv(output_file, index=False)

print(f"All folds processed and merged data saved successfully to {output_file}.")


Processed fold 1.
Processed fold 2.
Processed fold 3.
Processed fold 4.
Processed fold 5.
All folds processed and merged data saved successfully to xgboost_42_heating_residual.csv.


## Stacking Regressor

### hyper parameter tuning for heating Stacking Regressor

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Placeholder for storing results
results_stacking = []

experiment = 2  # Experiment number
# Define hyperparameter grid for Stacking Regressor
param_grid_stacking = {
    'final_estimator__fit_intercept': [True, False],  # Meta-model parameter (LinearRegression)
    'rf__n_estimators': [50, 100],                   # Base model: Random Forest
    'rf__max_depth': [10, 20],                       # Base model: Random Forest
    'gb__learning_rate': [0.01, 0.1],                # Base model: Gradient Boosting
    'gb__max_depth': [3, 6]                          # Base model: Gradient Boosting
}

for i in range(1, 6):
    # Load datasets
    train_data = pd.read_csv(f'fold{i}_heating_filtered_train_42_data.csv')
    test_data = pd.read_csv(f'fold{i}_heating_filtered_test_42_data.csv')

    # Split features and target
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Define base models and Stacking Regressor
    base_models = [
        ('rf', RandomForestRegressor(random_state=100)),
        ('gb', GradientBoostingRegressor(random_state=100))
    ]
    meta_model = LinearRegression()
    stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Perform GridSearchCV on the Stacking Regressor
    grid_search_stacking = GridSearchCV(stacking, param_grid_stacking, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search_stacking.fit(X_train, y_train)

    # Get the best model and hyperparameters
    best_stacking = grid_search_stacking.best_estimator_
    best_params = grid_search_stacking.best_params_

    # Split test features and target
    X_test = test_data.drop(columns=['WH_RTU_Total'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate metrics
    y_pred = best_stacking.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results_stacking.append({
        'Fold': i,
        'Seed': 42,
        'Model': 'Stacking Regressor',
        'Dataset': 'heating',
        'MAE': mae,
        'R²': r2,
        'Best Params': best_params,
        'Experiment': experiment
    })

    print(f"Fold {i}, Model Stacking Regressor, Dataset heating, MAE: {mae:.4f}, R²: {r2:.4f}, Best Params: {best_params}")

# Convert results to DataFrame
results_stacking_df = pd.DataFrame(results_stacking)

# Save detailed results to CSV
results_stacking_df.to_csv('stacking_42_heating_hyperparameter_results_detailed.csv', index=False)

# Display overall summary
print("\nSummary Results for Stacking Regressor:")
summary_stacking = results_stacking_df.groupby(['Model', 'Dataset']).agg({'MAE': 'mean', 'R²': 'mean'}).reset_index()
print(summary_stacking)

# Save summary to CSV
summary_stacking.to_csv('stacking_42_heating_summary_results.csv', index=False)


Fold 1, Model Stacking Regressor, Dataset heating, MAE: 2.7927, R²: 0.5717, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 20, 'rf__n_estimators': 100}
Fold 2, Model Stacking Regressor, Dataset heating, MAE: 2.8901, R²: 0.5801, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 20, 'rf__n_estimators': 50}
Fold 3, Model Stacking Regressor, Dataset heating, MAE: 3.2133, R²: 0.5485, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 20, 'rf__n_estimators': 100}
Fold 4, Model Stacking Regressor, Dataset heating, MAE: 3.0033, R²: 0.6176, Best Params: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__max_depth': 6, 'rf__max_depth': 10, 'rf__n_estimators': 50}
Fold 5, Model Stacking Regressor, Dataset heating, MAE: 2.8292, R²: 0.6159, Best Params: {'final_estimator__fi

### Feature Importance

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

# Placeholder for feature importance results
feature_importance_results = []


experiment = 2  # Experiment number

# Section 1: Calculate Feature Importance
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_heating_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Extract the best parameters from the results file
    results_stacking_df = pd.read_csv('stacking_42_heating_hyperparameter_results_detailed.csv')
    best_params = results_stacking_df.loc[
        (results_stacking_df['Fold'] == fold) & (results_stacking_df['Dataset'] == 'heating'),
        'Best Params'
    ].iloc[0]
    best_params_dict = eval(best_params)  # Parse string into dictionary

    # Define base models
    base_models = [
        ('rf', RandomForestRegressor(random_state=100, **{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('rf__')})),
        ('gb', GradientBoostingRegressor(random_state=100, **{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('gb__')}))
    ]
    meta_model = LinearRegression(**{k.split('__')[1]: v for k, v in best_params_dict.items() if k.startswith('final_estimator__')})

    # Define Stacking Regressor and fit the model
    stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)
    stacking.fit(X_train, y_train)

    # Calculate feature importances from base models
    for model_name, model in stacking.named_estimators_.items():
        if hasattr(model, 'feature_importances_'):  # Check if the model supports feature importance
            feature_importances = model.feature_importances_
            feature_names = X_train.columns
            importance_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': feature_importances,
                'Model': model_name,
                'Fold': fold,
                'Seed': 42,
                'Experiment': experiment  # Add experiment column
            })
            feature_importance_results.append(importance_df)

# Combine all feature importance results
feature_importance_combined = pd.concat(feature_importance_results, ignore_index=True)

# Save the feature importance results to a file
feature_importance_combined.to_csv(f'stacking_42_heating_feature_importance_experiment_{experiment}.csv', index=False)

print(f"Feature importance for seed 42 and experiment {experiment} has been successfully saved.")

Feature importance for seed 42 and experiment 2 has been successfully saved.


### Residual Analysis

In [ ]:
import pandas as pd
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression


experiment = 2  # Experiment number
merged_data = []  # Placeholder for storing final merged data

# Load the best model parameters for each fold
best_params_df = pd.read_csv('stacking_42_heating_hyperparameter_results_detailed.csv')

# Loop through each fold
for fold in range(1, 6):
    # Load training data
    train_data = pd.read_csv(f'fold{fold}_heating_filtered_train_42_data.csv')
    X_train = train_data.drop(columns=['WH_RTU_Total'])
    y_train = train_data['WH_RTU_Total']

    # Get the best parameters for the current fold
    best_params = best_params_df.loc[best_params_df['Fold'] == fold, 'Best Params'].iloc[0]
    best_params_dict = eval(best_params)

    # Define base models and Stacking Regressor with best parameters
    base_models = [
        ('rf', RandomForestRegressor(random_state=100,
                                     n_estimators=best_params_dict['rf__n_estimators'],
                                     max_depth=best_params_dict['rf__max_depth'])),
        ('gb', GradientBoostingRegressor(random_state=100,
                                         learning_rate=best_params_dict['gb__learning_rate'],
                                         max_depth=best_params_dict['gb__max_depth']))
    ]
    meta_model = LinearRegression(fit_intercept=best_params_dict['final_estimator__fit_intercept'])
    best_stacking = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Train the Stacking Regressor
    best_stacking.fit(X_train, y_train)

    # Load test data
    test_data = pd.read_csv(f'fold{fold}_heating_filtered_test_42_data.csv')
    test_data['Label'] = 'heating'  # Dynamically add label based on dataset type

    # Split features and target
    X_test = test_data.drop(columns=['WH_RTU_Total', 'Label'])
    y_test = test_data['WH_RTU_Total']

    # Predict and calculate residuals
    y_pred = best_stacking.predict(X_test)
    test_data = test_data.reset_index(drop=True)  # Reset test_data index
    y_pred_series = pd.Series(y_pred, index=test_data.index)  # Align y_pred with test_data index
    test_data['Predicted'] = y_pred_series
    test_data['Residual'] = test_data['WH_RTU_Total'] - test_data['Predicted']

    # Select and rename required columns
    if 'T_out' in test_data.columns and 'RH_out' in test_data.columns:
        selected_data = test_data[['WH_RTU_Total', 'Predicted', 'Residual', 'T_out', 'RH_out', 'Label']].rename(columns={
            'WH_RTU_Total': 'Actual'
        })
    else:
        missing_columns = [col for col in ['T_out', 'RH_out'] if col not in test_data.columns]
        raise ValueError(f"{', '.join(missing_columns)} is/are missing in test data for fold {fold}")

    # Add fold, seed, and model columns
    selected_data['Fold'] = fold
    selected_data['Seed'] = 42
    selected_data['Model'] = 'StackingRegressor'
    selected_data['Experiment'] = experiment

    # Append to final merged data
    merged_data.append(selected_data)
    print(f"Processed fold {fold}.")

# Combine all folds into a single DataFrame
merged_final_data = pd.concat(merged_data, ignore_index=True)

# Save the merged data
output_file = 'stacking_42_heating_residual.csv'
merged_final_data.to_csv(output_file, index=False)

print(f"All folds processed and merged data saved successfully to {output_file}.")


Processed fold 1.
Processed fold 2.
Processed fold 3.
Processed fold 4.
Processed fold 5.
All folds processed and merged data saved successfully to stacking_42_heating_residual.csv.
